In [4]:
import requests
import time

# get the current utc timestamp 
current_timestamp = int(time.time())
endpoint = "https://query2.finance.yahoo.com/v8/finance/chart/SWDA.MI"
params = {
    "period1": 1253862000,
    "period2":current_timestamp,
    "interval": "1d",
    "includePrePost": "true",
}
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}
r = requests.get(endpoint, params=params, headers=headers)
df  = r.json()

In [12]:
import pandas as pd

result = data['chart']['result'][0]
current_price = result['meta']['regularMarketPrice']
close = result['indicators']['quote'][0]['close']
ts = data['chart']['result'][0]['timestamp']

df = pd.DataFrame({'close': close, 'timestamp': ts})
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [10]:
df 

,close,timestamp
0,18.400000,2009-09-25 07:00:00
1,18.400000,2009-09-28 07:00:00
2,18.400000,2009-09-29 07:00:00
3,18.400000,2009-09-30 07:00:00
4,18.400000,2009-10-01 07:00:00
...,...,...
3851,102.379997,2024-11-18 08:00:00
3852,102.269997,2024-11-19 08:00:00
3853,102.370003,2024-11-20 08:00:00
3854,103.720001,2024-11-21 08:00:00


In [8]:
current_price

104.89

In [13]:
# Calculate the 20-period Simple Moving Average (SMA)
df['SMA'] = df['close'].rolling(window=20).mean()

# Calculate the 20-period Standard Deviation (SD)
df['SD'] = df['close'].rolling(window=20).std()

# Calculate the Upper Bollinger Band (UB) and Lower Bollinger Band (LB)
df['UB'] = df['SMA'] + 2 * df['SD']
df['LB'] = df['SMA'] - 2 * df['SD']

In [20]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column
import numpy as np

# Enable notebook output
output_notebook()

# Create a Bokeh figure
p = figure(width=10000, height=800, x_axis_type='datetime', title='SWDA.MI ETF with Bollinger Bands')

# Add the price line
p.line(df['timestamp'], df['close'], line_color='black', legend_label='Price', line_width=2)

# Add Bollinger Bands
p.line(df['timestamp'], df['UB'], line_color='red', legend_label='Upper Band', line_width=1)
p.line(df['timestamp'], df['LB'], line_color='green', legend_label='Lower Band', line_width=1)
p.line(df['timestamp'], df['SMA'], line_color='blue', legend_label='20-day MA', line_width=1)

# Add red dots for points below lower band
below_lb_mask = df['close'] < df['LB']
p.circle(df['timestamp'][below_lb_mask], df['close'][below_lb_mask], 
         size=8, color='red', legend_label='Below Lower Band')

# Fill between the bands
band_x = np.append(df['timestamp'], df['timestamp'][::-1])
band_y = np.append(df['UB'], df['LB'][::-1])
p.patch(band_x, band_y, color='gray', alpha=0.2, legend_label='Bollinger Bands')

# Customize the plot
p.grid.grid_line_alpha = 0.3
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Add axis labels
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'

# Show the plot
show(p)

Loading BokehJS ...